In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
from scipy import ndimage, sparse, linalg
from skimage.color import label2rgb
import pyrtools as pt
from tqdm import tqdm

from utils import im2double, gauss2D, local_min, get_weights

In [2]:
def get_speed_based_on_gradient(img, normSigma=5):
    [gx,gy] = np.gradient(255*img)
    mag = np.sqrt(gx**2 + gy**2)
    filt = np.array([1])
    filt = filt.reshape((1,-1))
    ss_mag = pt.corrDn(mag, filt,'repeat', step=[2, 2])
    stdev = normSigma
    nTaps = np.round(3*stdev) * 2 + 1
    
    lpImpResp = gauss2D((1,nTaps),stdev)
    lpImpResp = lpImpResp / np.max(lpImpResp)
    smooth_ssmag0 = ndimage.convolve(ss_mag, lpImpResp)
    smooth_ssmag = ndimage.convolve(smooth_ssmag0, lpImpResp.conj().T)

    f = np.array([0.5,1.0,0.5]).conj().T
    f = f.reshape(-1,1)
    res = pt.upConv(smooth_ssmag,f,'reflect1',step=[2,1])
    smooth_mag = pt.upConv(res,f.conj().T,'reflect1',step=[1,2])

    smooth_mag = smooth_mag / (np.sqrt(2*np.pi) * stdev)
    if (smooth_mag.shape[0] != mag.shape[0]):
        smooth_mag = smooth_mag[:-1,:]
    if (smooth_mag.shape[1] != mag.shape[1]):
        smooth_mag = smooth_mag[:,:-1]
    magHalfHeight = 10.0
    normGradMag = 127 * (mag / (magHalfHeight + smooth_mag))
    speed = np.exp(-normGradMag/10)
    Dx= np.exp(normGradMag/10)
    return Dx,speed

def get_seeds_orig(img,num_seeds,speed):
    size_grid = np.sqrt(np.size(img) / num_seeds)
    rows = img.shape[0] / size_grid
    cols = img.shape[1] / size_grid
    size_grid_row = img.shape[0] / math.ceil(rows)
    size_grid_col = img.shape[1] / math.ceil(cols)
    [x,y] = np.mgrid[0:math.ceil(rows-1)+1,0:math.ceil(cols-1)+1]
    x = x.astype(float).T
    y = y.astype(float).T
    y = y*size_grid_col + size_grid_col/2
    x = x*size_grid_row + size_grid_row/2
    
    mag = 1-speed
    
    minDistBetweenSeeds = min(size_grid_row,size_grid_col)
    seedRadius = 1
    
    maxShift = math.floor((minDistBetweenSeeds - 2*seedRadius) / 2) - 1
    N = math.ceil(maxShift/2)
#     size = (2*N, 2*N)
#     shape = cv2.MORPH_RECT
#     kernel = cv2.getStructuringElement(shape, size)
#     min_image = cv2.erode(mag, kernel)
    [dx,dy] = local_min(img,N)
    ind = np.hstack((x.astype("int")[0,:],y.astype("int")[:,0]))
    x_row = list(x[0,:].astype("int")-1)
    y_row = list(y[:,0].astype("int")-1)
    new_x = dx[np.ix_(x_row, y_row)].T
    new_y = dy[np.ix_(x_row, y_row)].T
    x = new_x
    y = new_y
    off = 2
    x_trun = x.copy()
    x_trun[x_trun>(img.shape[0]-off)] = img.shape[0]-off
    y_trun = y.copy()
    y_trun[y_trun>(img.shape[1]-off)] = img.shape[1]-off
    x = x_trun
    x[x<(1+off)] = 1+off
    y = y_trun.copy()
    y[y<(1+off)] = 1+off
    
    seeds = np.vstack((x.flatten(),y.flatten())).T
    return seeds   

def generate_seeds(Nsp,img):
    n_init = np.round(Nsp/4)
    expSuperPixelDist = np.sqrt(np.size(img)/n_init)
    normSigma = math.floor(expSuperPixelDist / 2.5)
    _,speed = get_speed_based_on_gradient(img,normSigma)
#     print ("speed:",speed[0,:])
    seeds = get_seeds_orig(img,n_init,speed)
    return seeds

In [3]:
# Params
Nsp=200 #num of sp
Thres=1.35 #threshold for split
beta=30 #gaussian parameter 
alpha=0.9992 #Lazy parameter
nItrs_max = 10 #limit for the number of iterations

In [4]:
img = cv2.cvtColor(cv2.imread("../images/290.jpg"),cv2.COLOR_BGR2RGB)
scale_percent = 25
width = math.ceil(img.shape[1] * scale_percent / 100)
height = math.ceil(img.shape[0] * scale_percent / 100)
dim = (width, height)
  
img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
gray_img = cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2GRAY)
orig_img = img.copy()
img = im2double(img)
X,Y,Z = img.shape

In [5]:
seeds = generate_seeds(Nsp,im2double(gray_img/255))
print(seeds[:5])

[[ 4.  4.]
 [22.  5.]
 [35.  5.]
 [45.  5.]
 [62.  6.]]


In [6]:
def LRW(adj, seeds, labels, alpha, size):
    """Find and optimize the superpixel results
    Args:
        adj: The adjacency matrix of the graph generated from image
        seeds: The seed positions (Row major form)
        labels: The labels corrosponding to the seeds
        alpha: Probability of staying put in LRW
        size: Size of the image
    Returns:
        image: Labeled image
        prob: The probability that the seed belongs to the label
    """
    I = sparse.identity(size)
    D_inv = sparse.coo_matrix((np.asarray(adj.sum(axis=0)).ravel(), 
                              (np.arange(size), np.arange(size))), 
                              shape=(size, size))
    np.reciprocal(D_inv.data, out=D_inv.data)
    lines = np.zeros(shape=(size, len(labels)))
    for k in range(len(seeds)):
        label_idx = np.where(labels == k)[0]
        mk = label_idx.shape[0]
        lines[seeds[label_idx].astype(int), k] = 1 / mk
    D_inv_sqrt = D_inv.sqrt()
    S = D_inv_sqrt @ adj @ D_inv_sqrt
    print(S.shape)
    print(I - alpha*S)
    sparse.linalg.inv(S)
    # flk = sparse.linalg.inv(I - alpha*S);
        

def energy_opt(image, seeds, alpha, count, iters, sigma, thres):
    """Find and optimize the superpixel results
    Args:
        image: Original Image (RGB / Grayscale)
        seeds: The initial seed positions
        alpha: Probability of staying put in LRW
        count: Number of superpixels
        iters: Max number of iterations
        sigma: Gaussian parameter
        thres: Threshold to split bigger superpixels
    Returns:
        label: Labeled image
        seeds: The optimized seed positions
        iters: The number of iterations taken for convergence
    """

    height, width = image.shape[:2]
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)

    # Generate adjacency matrix
    adj = get_weights(lab, sigma)          
    
    # Iteratvely improve superpixels
    new_seeds = seeds.copy()
    iter_num = 0
    while len(new_seeds) < count and iter_num < iters:
        # Get labels and seed indices in row major form
        seeds_idx = new_seeds[:,0]*width + new_seeds[:,1]
        labels = np.arange(len(new_seeds))
        LRW(adj, new_seeds, labels, alpha, height*width)
        break
        iter_num += 1
    
    return [], new_seeds, iter_num

res = energy_opt(orig_img, seeds, alpha, Nsp, nItrs_max, beta, Thres)

(9801, 9801)
  (0, 0)	1.0
  (0, 1)	-0.48794577864249866
  (0, 121)	-0.3587758507364876
  (1, 0)	-0.48794577864249866
  (1, 1)	1.0
  (1, 2)	-0.20982123991727944
  (1, 122)	-0.3105673876177914
  (2, 1)	-0.2098212399172794
  (2, 2)	1.0
  (2, 3)	-0.4389172732326129
  (2, 123)	-0.392995369643245
  (3, 2)	-0.438917273232613
  (3, 3)	1.0
  (3, 4)	-0.27339613440609617
  (3, 124)	-0.26048289257951635
  (4, 3)	-0.27339613440609617
  (4, 4)	1.0
  (4, 5)	-0.3319652111394065
  (4, 125)	-0.3016407249205187
  (5, 4)	-0.3319652111394066
  (5, 5)	1.0
  (5, 6)	-0.3174306345477073
  (5, 126)	-0.26694913033132994
  (6, 5)	-0.3174306345477073
  (6, 6)	1.0
  :	:
  (9794, 9794)	1.0
  (9794, 9795)	-0.33005991440546406
  (9795, 9674)	-0.02141581511970462
  (9795, 9794)	-0.33005991440546406
  (9795, 9795)	1.0
  (9795, 9796)	-0.43688700794491137
  (9796, 9675)	-0.017835322204174133
  (9796, 9795)	-0.43688700794491137
  (9796, 9796)	1.0
  (9796, 9797)	-0.6095865578917624
  (9797, 9676)	-0.007386268076275087
  (97

In [7]:
a = np.array([0, 2, 3, 5, 6])
np.where(a > 3)[0]

array([3, 4])